In [19]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from torch_geometric.data import Data,Dataset

total_parms_new = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params_new.csv',index_col=0)
total_parms = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/total_params.csv',index_col=0)

from typing import List, Literal, Sequence, Union


dataset_list = ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']

def get_datasetwise_df(df:pd.DataFrame,name=None):
    if not name:
        dflist = []
        for i in ['cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia']:
            dflist.append(df[df['Dataset']==i])
        return dflist
    else:
        return df[df['Dataset']==name]

def baseline(df:pd.DataFrame):
    clean_adam = df[(df['Dp']==False)&(df['Rdp']==True)&(df['Ldp']==False)&(df['Optim type']=='adam')]
    clean_sgd = df[(df['Dp']==False)&(df['Rdp']==True)&(df['Ldp']==False)&(df['Optim type']=='sgd')]
    return clean_adam,clean_sgd

def rdp_drop(df:pd.DataFrame):
    labels = ['Vanilla train acc','Vanilla train loss','Vanilla train f1',
          'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate','MIA seed','Shadow runtime per', 'Num val', 'Num test', 'Layers', 'Hidden dims','Activation','Shadow learning rate','Dropout','Sampler','Dp','Rdp','Ldp']
    return df.drop(columns=labels)

def get_res_rdp(cora:pd.DataFrame):
    maxx = ['Vanilla test acc','Vanilla test f1']
    meann = ['Vanilla runtime per','MIA mlp', 'MIA svm','MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse','Epsilon','Delta']
    minn = ['Epsilon','Delta']
    res = {}
   
    for k in maxx:
        res[k] = cora.max()[k]
    for k in meann:
        res[k] = cora.mean(numeric_only=True)[k]
    for k in minn:
        res[k] = cora.mean(numeric_only=True)[k]
    return res
    

total_parms_new.columns

Index(['Dataset', 'Vanilla train acc', 'Vanilla train loss',
       'Vanilla train f1', 'Vanilla test acc', 'Vanilla test loss',
       'Vanilla test f1', 'Shadow train acc', 'Shadow train loss',
       'Shadow train f1', 'Shadow test acc', 'Shadow test loss',
       'Shadow test f1', 'MIA subsample rate', 'MIA mlp', 'MIA svm',
       'MIA ranfor', 'MIA logi', 'MIA ada', 'MIA confidence mse',
       'MIA confidence thr', 'MIA seed', 'Vanilla runtime per',
       'Shadow runtime per', 'Epsilon', 'Delta', 'Dp', 'Rdp', 'Ldp',
       'Norm bound', 'Noise scale', 'Sampler', 'Sampler batchsize',
       'Occurance k', 'Cluster numparts', 'Saint rootnodes',
       'Saint samplecoverage', 'Saint walklenth', 'Epochs', 'Shadow epochs',
       'Num val', 'Num test', 'Layers', 'Hidden dims', 'Learning rate',
       'Shadow learning rate', 'Dropout', 'Activation', 'Early stopping',
       'Patience', 'Optim type'],
      dtype='object')

'cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia'

In [20]:
dfs = get_datasetwise_df(total_parms_new)
cora = dfs[10]
cora_adam,cora_sgd = baseline(cora)
cora_adam['Sampler'].drop_duplicates()

0    occurance
Name: Sampler, dtype: object

In [21]:
cora_adam_d = cora_adam[cora_adam['Sampler']=='saint_rw']
rdp_drop(cora_adam_d)['Sampler batchsize'].drop_duplicates()

Series([], Name: Sampler batchsize, dtype: float64)

saint

In [22]:
cora_adam_d = cora_adam[cora_adam['Sampler']=='saint_rw']
rdp_drop(cora_adam_d)['Sampler batchsize'].drop_duplicates()

Series([], Name: Sampler batchsize, dtype: float64)

In [23]:
get_res_rdp(cora_adam_d[cora_adam_d['Noise scale']==1])

{'Vanilla test acc': nan,
 'Vanilla test f1': nan,
 'Vanilla runtime per': nan,
 'MIA mlp': nan,
 'MIA svm': nan,
 'MIA ranfor': nan,
 'MIA logi': nan,
 'MIA ada': nan,
 'MIA confidence mse': nan,
 'Epsilon': nan,
 'Delta': nan}

In [24]:
df = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/cora_ns2_nb1_saint_rw.csv',index_col=0)
df['Sampler batchsize'].drop_duplicates()

0    0.4
Name: Sampler batchsize, dtype: float64

In [25]:
get_res_rdp(df)

{'Vanilla test acc': 0.8062,
 'Vanilla test f1': 0.8062,
 'Vanilla runtime per': 11.112649999999999,
 'MIA mlp': 0.5311999999999999,
 'MIA svm': 0.5681499999999999,
 'MIA ranfor': 0.52425,
 'MIA logi': 0.5234,
 'MIA ada': 0.5197,
 'MIA confidence mse': 0.59565,
 'Epsilon': 1.2177,
 'Delta': 0.0}

Cluster

In [26]:
df = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/fli_ns2_nb1_cluster.csv',index_col=0)
df['Sampler batchsize'].drop_duplicates()

0    0.4
1    0.2
Name: Sampler batchsize, dtype: float64

In [27]:
df['Cluster numparts'].drop_duplicates()

0     30.0
1    100.0
Name: Cluster numparts, dtype: float64

In [28]:
get_res_rdp(df[(df['Sampler batchsize']==.2)&(df['Cluster numparts']==100)])

{'Vanilla test acc': 0.4311,
 'Vanilla test f1': 0.4311,
 'Vanilla runtime per': 3.19445,
 'MIA mlp': 0.50085,
 'MIA svm': 0.49925,
 'MIA ranfor': 0.50175,
 'MIA logi': 0.50575,
 'MIA ada': 0.4974,
 'MIA confidence mse': 0.52015,
 'Epsilon': 19.50705,
 'Delta': 0.001}

In [29]:
df = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/git_ns2_nb1_cluster.csv',index_col=0)
df['Sampler batchsize'].drop_duplicates()

0    0.4
Name: Sampler batchsize, dtype: float64

In [30]:
df['Cluster numparts'].drop_duplicates()

0    30.0
Name: Cluster numparts, dtype: float64

In [31]:
get_res_rdp(df)

{'Vanilla test acc': 0.8532,
 'Vanilla test f1': 0.8532,
 'Vanilla runtime per': 1.6186700000000003,
 'MIA mlp': 0.50057,
 'MIA svm': 0.5004099999999999,
 'MIA ranfor': 0.50175,
 'MIA logi': 0.5005799999999999,
 'MIA ada': 0.5005,
 'MIA confidence mse': 0.5065599999999999,
 'Epsilon': 45.85263,
 'Delta': 0.0033}

'cora','citeseer','pubmed','computers','photo','cs','physics','reddit','github','flickr','lastfmasia'

In [32]:
dfs = get_datasetwise_df(total_parms_new)
cora = dfs[10]
cora_adam,cora_sgd = baseline(cora)
cora_adam['Sampler'].drop_duplicates()


0    occurance
Name: Sampler, dtype: object

In [33]:
cora_adam_d = cora_adam[cora_adam['Sampler']=='saint_rw']
rdp_drop(cora_adam_d)['Sampler batchsize'].drop_duplicates()

Series([], Name: Sampler batchsize, dtype: float64)

In [34]:
cora_adam_d = cora_adam[cora_adam['Sampler']=='cluster']
rdp_drop(cora_adam_d)['Cluster numparts'].drop_duplicates()

Series([], Name: Cluster numparts, dtype: float64)

In [35]:
cora_adam_d = cora_adam[cora_adam['Sampler']=='cluster']
rdp_drop(cora_adam_d)['Sampler batchsize'].drop_duplicates()

Series([], Name: Sampler batchsize, dtype: float64)

In [36]:
cora_adam_d = cora_adam[cora_adam['Sampler']=='cluster']
rdp_drop(cora_adam_d)['Noise scale'].drop_duplicates()

Series([], Name: Noise scale, dtype: float64)

In [37]:
cora_adam_d = cora_adam[cora_adam['Sampler']=='cluster']
rdp_drop(cora_adam_d)['Norm bound'].drop_duplicates()

Series([], Name: Norm bound, dtype: float64)

In [38]:
cora_sb2_ns1_nb1 = cora_adam_d[(cora_adam_d['Norm bound']==1)&(cora_adam_d['Noise scale']==1)&(cora_adam_d['Sampler batchsize']==.4)]

In [39]:
cora_sb2_ns1_nb1

,Dataset,Vanilla train acc,Vanilla train loss,Vanilla train f1,Vanilla test acc,Vanilla test loss,Vanilla test f1,Shadow train acc,Shadow train loss,Shadow train f1,...,Num test,Layers,Hidden dims,Learning rate,Shadow learning rate,Dropout,Activation,Early stopping,Patience,Optim type


In [40]:
get_res_rdp(cora_sb2_ns1_nb1)

{'Vanilla test acc': nan,
 'Vanilla test f1': nan,
 'Vanilla runtime per': nan,
 'MIA mlp': nan,
 'MIA svm': nan,
 'MIA ranfor': nan,
 'MIA logi': nan,
 'MIA ada': nan,
 'MIA confidence mse': nan,
 'Epsilon': nan,
 'Delta': nan}

In [41]:
df=pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/cora_ns2_nb1_cluster.csv',index_col=0)
df['Sampler batchsize'].drop_duplicates()

0    0.4
Name: Sampler batchsize, dtype: float64

In [42]:
get_res_rdp(df[df['Sampler batchsize']==.4])

{'Vanilla test acc': 0.7488,
 'Vanilla test f1': 0.7488,
 'Vanilla runtime per': 0.7607799999999999,
 'MIA mlp': 0.50417,
 'MIA svm': 0.5147,
 'MIA ranfor': 0.50132,
 'MIA logi': 0.51298,
 'MIA ada': 0.50139,
 'MIA confidence mse': 0.58505,
 'Epsilon': 47.9038,
 'Delta': 0.0033}

In [43]:
df = pd.read_csv('/mnt/ssd1/sunyifan/WorkStation/dpuf/my_DPGCN/ress/cora_ns1_nb1_neighbor.csv',index_col=0)
get_res_rdp(df)

{'Vanilla test acc': 0.8391,
 'Vanilla test f1': 0.8391,
 'Vanilla runtime per': 0.76777,
 'MIA mlp': 0.5893200000000001,
 'MIA svm': 0.60196,
 'MIA ranfor': 0.5478799999999999,
 'MIA logi': 0.5133,
 'MIA ada': 0.55107,
 'MIA confidence mse': 0.62348,
 'Epsilon': 164.0745,
 'Delta': 0.0002}